### 항공편 데이터를 통해서 머신러닝 모델에 적용하기

In [317]:
# 데이터 가져오기

import pandas as pd
import numpy as np

data = pd.read_csv("airportal.csv").drop(columns = ["예상", "구분"])        ## 애당초 필요 없는 columns 삭제 후 read

data.head()

,연월일,항공사,편명,도착지,계획,실제,현황
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05,출발
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01,출발
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51,출발
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18,출발
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54,지연


In [318]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179719 entries, 0 to 179718
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   연월일     179719 non-null  object
 1   항공사     179719 non-null  object
 2   편명      179719 non-null  object
 3   도착지     179719 non-null  object
 4   계획      179719 non-null  object
 5   실제      179719 non-null  object
 6   현황      179719 non-null  object
dtypes: object(7)
memory usage: 9.6+ MB


* 어쩐 일인지 결측치가 없는 것 같다.

##### 그렇다면, ["계획", "실제"]를 datetime 유형으로 바꾸기

In [288]:
# pd.to_datetime(data["실제"], format ='%H:%M')

## 돌리면 899번째 행에서 문제가 생겼다는 결과가 표시됨

* 899번째 행에서 문제가 생겼음을 확인

In [319]:
## 데이터 이상치 확인
data.iloc[899]

연월일    2024-01-02
항공사          대한항공
편명          KE719
도착지      HND(하네다)
계획          20:40
실제              :
현황             취소
Name: 899, dtype: object

##### 두 가지를 알 수 있다.
- 일단 현황에 "출발, 지연"만 있는 것은 아님.
- ":"만 있는 경우도 있음을 확인.

In [320]:
# 일단 "현황"에 어떠한 값이 있는지 확인
data["현황"].unique()

array(['출발', '지연', '취소', '회항'], dtype=object)

* 그렇다면 취소와 회항은 얼마나 있나?

In [321]:
data["현황"].value_counts()

현황
출발    116470
지연     62816
취소       396
회항        37
Name: count, dtype: int64

* 취소와 회항 건의 차이가 출발과 지연과 차이가 심하므로, 해당 값은 누락시킴 ㅎ

In [322]:
## 데이터에서 "현황" 열이 출발이거나 지연인 데이터만 가져오기
data2 = data.loc[(data["현황"] == "출발") | (data["현황"] == "지연")].reset_index(drop = True)
data2.head()

,연월일,항공사,편명,도착지,계획,실제,현황
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05,출발
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01,출발
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51,출발
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18,출발
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54,지연


In [323]:
# ":"인 데이터들 정리
data3 = data2.loc[(data2["실제"] != ":") & (data2["계획"] != ":")].reset_index(drop = True)
data3.head()

,연월일,항공사,편명,도착지,계획,실제,현황
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05,출발
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01,출발
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51,출발
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18,출발
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54,지연


In [324]:
### 다시 현황을 drop
data4 = data3.drop("현황", axis = 1)

In [325]:
data4

,연월일,항공사,편명,도착지,계획,실제
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54
...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,SIN(싱가포르),23:10,23:33
179178,2024-12-31,싱가폴항공,SQ605,SIN(싱가포르),23:15,23:48
179179,2024-12-31,청도항공,QW9902,TAO(청도),23:20,23:26
179180,2024-12-31,필리핀에어아시아,Z2889,MNL(마닐라),23:40,23:51


In [ ]:
# 일단 후에 계획 날짜 및 실제 날짜를 통하여 날씨와의 연결성을 만들기 위해 string으로 적어줌.
data4["계획 날짜"] = data4["연월일"]+" "+data4["계획"]
data4["실제 날짜"] = data4["연월일"]+" "+data4["실제"]

In [330]:
data4

,연월일,항공사,편명,도착지,계획,실제,계획 날짜,실제 날짜
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05,2024-01-01 00:05,2024-01-01 00:05
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01,2024-01-01 00:05,2024-01-01 00:01
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51,2024-01-01 00:15,2024-01-01 00:51
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18,2024-01-01 00:15,2024-01-01 00:18
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54,2024-01-01 00:20,2024-01-01 01:54
...,...,...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,SIN(싱가포르),23:10,23:33,2024-12-31 23:10,2024-12-31 23:33
179178,2024-12-31,싱가폴항공,SQ605,SIN(싱가포르),23:15,23:48,2024-12-31 23:15,2024-12-31 23:48
179179,2024-12-31,청도항공,QW9902,TAO(청도),23:20,23:26,2024-12-31 23:20,2024-12-31 23:26
179180,2024-12-31,필리핀에어아시아,Z2889,MNL(마닐라),23:40,23:51,2024-12-31 23:40,2024-12-31 23:51


In [358]:
## 일단 이 상태에서 datetime으로 계획과 실제의 값을 변경
from datetime import datetime
data4["계획 날짜"] = pd.to_datetime(data4["계획 날짜"], format = "%Y-%m-%d %H:%M")
data4["실제 날짜"] = pd.to_datetime(data4["실제 날짜"], format = "%Y-%m-%d %H:%M")

data4

,연월일,항공사,편명,도착지,계획,실제,계획 날짜,실제 날짜,지연 시간,지연 여부
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05,2024-01-01 00:05:00,2024-01-01 00:05:00,0,0
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01,2024-01-01 00:05:00,2024-01-01 00:01:00,-4,0
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51,2024-01-01 00:15:00,2024-01-01 00:51:00,36,1
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18,2024-01-01 00:15:00,2024-01-01 00:18:00,3,0
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54,2024-01-01 00:20:00,2024-01-01 01:54:00,94,1
...,...,...,...,...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,SIN(싱가포르),23:10,23:33,2024-12-31 23:10:00,2024-12-31 23:33:00,23,1
179178,2024-12-31,싱가폴항공,SQ605,SIN(싱가포르),23:15,23:48,2024-12-31 23:15:00,2024-12-31 23:48:00,33,1
179179,2024-12-31,청도항공,QW9902,TAO(청도),23:20,23:26,2024-12-31 23:20:00,2024-12-31 23:26:00,6,0
179180,2024-12-31,필리핀에어아시아,Z2889,MNL(마닐라),23:40,23:51,2024-12-31 23:40:00,2024-12-31 23:51:00,11,0


In [359]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179182 entries, 0 to 179181
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   연월일     179182 non-null  object        
 1   항공사     179182 non-null  object        
 2   편명      179182 non-null  object        
 3   도착지     179182 non-null  object        
 4   계획      179182 non-null  object        
 5   실제      179182 non-null  object        
 6   계획 날짜   179182 non-null  datetime64[ns]
 7   실제 날짜   179182 non-null  datetime64[ns]
 8   지연 시간   179182 non-null  int64         
 9   지연 여부   179182 non-null  int64         
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 13.7+ MB


In [360]:
## 이 형태를 기반으로 시간 차이 계산하기
diff_list = []
for i in range(len(data4.index)):
    plan = data4["계획 날짜"][i]
    real = data4["실제 날짜"][i]
    if plan >= real :
        diff = int(-(plan - real).seconds / 60)
        diff_list.append(diff)
    else:
        diff = int((real - plan).seconds / 60)
        diff_list.append(diff)

data4["지연 시간"] = diff_list


In [361]:
# 벡터화 연산을 통한 빠른 계산 -> 지연 여부를 파악하기 위함
data4['지연 여부'] = (data4['지연 시간'] >= 16).astype(int)         ### 연산 시 전체 배열에 대한 불리언을 정수형으로 반환 -> 속도가 더 빠름

# 원래 구현 코드 : data4["지연 여부"] = data4["지연 날짜"].apply(lambda x: 1 if x >= 16 else 0)
### 원래 구현 코드는 Python 인터프리터를 통해 반복적으로 함수를 호출하는 방식이기에, 그 속도가 위 벡터화 연산에 비해 느림

In [362]:
data4

,연월일,항공사,편명,도착지,계획,실제,계획 날짜,실제 날짜,지연 시간,지연 여부
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),00:05,00:05,2024-01-01 00:05:00,2024-01-01 00:05:00,0,0
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),00:05,00:01,2024-01-01 00:05:00,2024-01-01 00:01:00,-4,0
2,2024-01-01,카타르항공,QR859,DOH(도하),00:15,00:51,2024-01-01 00:15:00,2024-01-01 00:51:00,36,1
3,2024-01-01,터키항공,TK91,IST(이스탄불),00:15,00:18,2024-01-01 00:15:00,2024-01-01 00:18:00,3,0
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),00:20,01:54,2024-01-01 00:20:00,2024-01-01 01:54:00,94,1
...,...,...,...,...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,SIN(싱가포르),23:10,23:33,2024-12-31 23:10:00,2024-12-31 23:33:00,23,1
179178,2024-12-31,싱가폴항공,SQ605,SIN(싱가포르),23:15,23:48,2024-12-31 23:15:00,2024-12-31 23:48:00,33,1
179179,2024-12-31,청도항공,QW9902,TAO(청도),23:20,23:26,2024-12-31 23:20:00,2024-12-31 23:26:00,6,0
179180,2024-12-31,필리핀에어아시아,Z2889,MNL(마닐라),23:40,23:51,2024-12-31 23:40:00,2024-12-31 23:51:00,11,0


In [ ]:
# '분'을 내림처리하기 위해서, apply를 통해 각 셀 별 minute = 0으로 바꿔주는 함수 입력 -> lambda를 통해 내부의 값을 바로 변환환
data4["실제 날짜"] = data4["실제 날짜"].apply(lambda x: x.replace(minute = 0))

0        2024-01-01 00:00:00
1        2024-01-01 00:00:00
2        2024-01-01 00:00:00
3        2024-01-01 00:00:00
4        2024-01-01 01:00:00
                 ...        
179177   2024-12-31 23:00:00
179178   2024-12-31 23:00:00
179179   2024-12-31 23:00:00
179180   2024-12-31 23:00:00
179181   2024-12-31 23:00:00
Name: 실제 날짜, Length: 179182, dtype: datetime64[ns]

In [364]:
# 연월일 구분
data4[["year", "month", "day"]] = data4["연월일"].apply(lambda x: pd.Series(x.split('-')))

## 이후 월과 날짜에 따라 어떠한 연관성이 있는지를 보기 위함

In [365]:
# 기상 데이터와 합치기 위해서 해당 값을 준비
year_time = []
for i in range(len(data4.index)):
    date_p = data4["연월일"][i].split("-")
    time_p = str(data4["실제 날짜짜"][i])[11:].split(":")
    year_time.append("".join(date_p)+"".join(time_p))

data4["YYMMDDHH"] = year_time


In [303]:
data4

,연월일,항공사,편명,도착지,계획,실제,지연 시간,지연 여부,year,month,day,YYMMDDHH
0,2024-01-01,에미레이트항공,EK323,DXB(두바이),1900-01-01 00:05:00,1900-01-01 00:05:00,0,0,2024,01,01,20240101000500
1,2024-01-01,몽골항공,OM308,UBN(칭기즈 칸(신 울란바토르) 국제공항),1900-01-01 00:05:00,1900-01-01 00:01:00,-4,0,2024,01,01,20240101000100
2,2024-01-01,카타르항공,QR859,DOH(도하),1900-01-01 00:15:00,1900-01-01 00:51:00,36,1,2024,01,01,20240101005100
3,2024-01-01,터키항공,TK91,IST(이스탄불),1900-01-01 00:15:00,1900-01-01 00:18:00,3,0,2024,01,01,20240101001800
4,2024-01-01,젯스타 에어웨이즈,JQ48,SYD(시드니),1900-01-01 00:20:00,1900-01-01 01:54:00,94,1,2024,01,01,20240101015400
...,...,...,...,...,...,...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,SIN(싱가포르),1900-01-01 23:10:00,1900-01-01 23:33:00,23,1,2024,12,31,20241231233300
179178,2024-12-31,싱가폴항공,SQ605,SIN(싱가포르),1900-01-01 23:15:00,1900-01-01 23:48:00,33,1,2024,12,31,20241231234800
179179,2024-12-31,청도항공,QW9902,TAO(청도),1900-01-01 23:20:00,1900-01-01 23:26:00,6,0,2024,12,31,20241231232600
179180,2024-12-31,필리핀에어아시아,Z2889,MNL(마닐라),1900-01-01 23:40:00,1900-01-01 23:51:00,11,0,2024,12,31,20241231235100


In [304]:
# 도착지 구분
data4[["도착지_코드", "도착지_한글", "도착지_코드2"]] = data4["도착지"].apply(lambda x: pd.Series(x.split('(')))
new_df = data4.drop(["도착지", "도착지_코드2"], axis = 1)

In [305]:
new_df

,연월일,항공사,편명,계획,실제,지연 시간,지연 여부,year,month,day,YYMMDDHH,도착지_코드,도착지_한글
0,2024-01-01,에미레이트항공,EK323,1900-01-01 00:05:00,1900-01-01 00:05:00,0,0,2024,01,01,20240101000500,DXB,두바이)
1,2024-01-01,몽골항공,OM308,1900-01-01 00:05:00,1900-01-01 00:01:00,-4,0,2024,01,01,20240101000100,UBN,칭기즈 칸
2,2024-01-01,카타르항공,QR859,1900-01-01 00:15:00,1900-01-01 00:51:00,36,1,2024,01,01,20240101005100,DOH,도하)
3,2024-01-01,터키항공,TK91,1900-01-01 00:15:00,1900-01-01 00:18:00,3,0,2024,01,01,20240101001800,IST,이스탄불)
4,2024-01-01,젯스타 에어웨이즈,JQ48,1900-01-01 00:20:00,1900-01-01 01:54:00,94,1,2024,01,01,20240101015400,SYD,시드니)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,1900-01-01 23:10:00,1900-01-01 23:33:00,23,1,2024,12,31,20241231233300,SIN,싱가포르)
179178,2024-12-31,싱가폴항공,SQ605,1900-01-01 23:15:00,1900-01-01 23:48:00,33,1,2024,12,31,20241231234800,SIN,싱가포르)
179179,2024-12-31,청도항공,QW9902,1900-01-01 23:20:00,1900-01-01 23:26:00,6,0,2024,12,31,20241231232600,TAO,청도)
179180,2024-12-31,필리핀에어아시아,Z2889,1900-01-01 23:40:00,1900-01-01 23:51:00,11,0,2024,12,31,20241231235100,MNL,마닐라)


In [306]:
# 괄호 정리
new_df["도착지_한글"] = new_df['도착지_한글'].str.replace(")", "")
new_df

,연월일,항공사,편명,계획,실제,지연 시간,지연 여부,year,month,day,YYMMDDHH,도착지_코드,도착지_한글
0,2024-01-01,에미레이트항공,EK323,1900-01-01 00:05:00,1900-01-01 00:05:00,0,0,2024,01,01,20240101000500,DXB,두바이
1,2024-01-01,몽골항공,OM308,1900-01-01 00:05:00,1900-01-01 00:01:00,-4,0,2024,01,01,20240101000100,UBN,칭기즈 칸
2,2024-01-01,카타르항공,QR859,1900-01-01 00:15:00,1900-01-01 00:51:00,36,1,2024,01,01,20240101005100,DOH,도하
3,2024-01-01,터키항공,TK91,1900-01-01 00:15:00,1900-01-01 00:18:00,3,0,2024,01,01,20240101001800,IST,이스탄불
4,2024-01-01,젯스타 에어웨이즈,JQ48,1900-01-01 00:20:00,1900-01-01 01:54:00,94,1,2024,01,01,20240101015400,SYD,시드니
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179177,2024-12-31,대한항공,KE647,1900-01-01 23:10:00,1900-01-01 23:33:00,23,1,2024,12,31,20241231233300,SIN,싱가포르
179178,2024-12-31,싱가폴항공,SQ605,1900-01-01 23:15:00,1900-01-01 23:48:00,33,1,2024,12,31,20241231234800,SIN,싱가포르
179179,2024-12-31,청도항공,QW9902,1900-01-01 23:20:00,1900-01-01 23:26:00,6,0,2024,12,31,20241231232600,TAO,청도
179180,2024-12-31,필리핀에어아시아,Z2889,1900-01-01 23:40:00,1900-01-01 23:51:00,11,0,2024,12,31,20241231235100,MNL,마닐라


In [307]:
# 사용하지 않은 columns 없애기
new_df = new_df.drop(["연월일", "계획", "실제", "year"], axis = 1)

In [308]:
new_df

,항공사,편명,지연 시간,지연 여부,month,day,YYMMDDHH,도착지_코드,도착지_한글
0,에미레이트항공,EK323,0,0,01,01,20240101000500,DXB,두바이
1,몽골항공,OM308,-4,0,01,01,20240101000100,UBN,칭기즈 칸
2,카타르항공,QR859,36,1,01,01,20240101005100,DOH,도하
3,터키항공,TK91,3,0,01,01,20240101001800,IST,이스탄불
4,젯스타 에어웨이즈,JQ48,94,1,01,01,20240101015400,SYD,시드니
...,...,...,...,...,...,...,...,...,...
179177,대한항공,KE647,23,1,12,31,20241231233300,SIN,싱가포르
179178,싱가폴항공,SQ605,33,1,12,31,20241231234800,SIN,싱가포르
179179,청도항공,QW9902,6,0,12,31,20241231232600,TAO,청도
179180,필리핀에어아시아,Z2889,11,0,12,31,20241231235100,MNL,마닐라


In [309]:
# 데이터 저장하기
new_df.to_csv("airportal_final.csv", index = False)

##### 이를 바탕으로 따로 조사하여 거리를 맞춘 "distance" column 추가, 각 순서를 정리하여 "airportal_final_new.csv" 파일로 만듦.

In [310]:
new_data = pd.read_csv("airportal_final_new.csv")
new_data

,airline,fight_code,month,day,YYMMDD,arrival_code,arrival_kor,distance(km),dealy_time,delayed
0,에미레이트항공,EK323,1,1,20240101000500,DXB,두바이,6836,0,0
1,몽골항공,OM308,1,1,20240101000100,UBN,칭기즈 칸,1995,-4,0
2,카타르항공,QR859,1,1,20240101005100,DOH,도하,7270,36,1
3,터키항공,TK91,1,1,20240101001800,IST,이스탄불,8055,3,0
4,젯스타 에어웨이즈,JQ48,1,1,20240101015400,SYD,시드니,8320,94,1
...,...,...,...,...,...,...,...,...,...,...
179177,대한항공,KE647,12,31,20241231233300,SIN,싱가포르,4630,23,1
179178,싱가폴항공,SQ605,12,31,20241231234800,SIN,싱가포르,4630,33,1
179179,청도항공,QW9902,12,31,20241231232600,TAO,청도,620,6,0
179180,필리핀에어아시아,Z2889,12,31,20241231235100,MNL,마닐라,2610,11,0


##### 날씨 데이터와 합치기!

In [311]:
# 기상 정보 파일 먼저 불러오기
weather_df = pd.read_csv("weather_data.csv")

In [312]:
weather_df

,YYMMDDHHMI,STN,WD,WS,GST,VS,RVR1,RVR2,WC,CA,...,CH3,CA4,CT4,CH4,TA,TD,HM,PA,PS,RN
0,202401010000,113,60,5,-9,5000,-9,-9,10,0,...,-9,-9,-9,-9,-0.7,-1.6,9.4,1026.0,1026.9,-9.0
1,202401010100,113,30,4,-9,4000,-9,-9,10,2,...,-9,-9,-9,-9,0.3,-1.0,9.1,1026.8,1027.6,-9.0
2,202401010200,113,50,4,-9,3500,-9,-9,10,1,...,-9,-9,-9,-9,0.1,-1.1,9.2,1026.9,1027.8,-9.0
3,202401010300,113,50,4,-9,2500,-9,-9,10,1,...,-9,-9,-9,-9,0.2,-1.1,9.1,1027.4,1028.3,-9.0
4,202401010400,113,50,6,-9,3000,-9,-9,10,7,...,-9,-9,-9,-9,0.1,-0.8,9.4,1027.5,1028.4,-9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,202412311900,113,280,8,-9,10000,-9,-9,-9,0,...,-9,-9,-9,-9,-1.1,-8.5,5.7,1022.3,1023.2,-9.0
8780,202412312000,113,230,1,-9,10000,-9,-9,-9,0,...,-9,-9,-9,-9,0.2,-7.7,5.5,1022.5,1023.4,-9.0
8781,202412312100,113,300,7,-9,10000,-9,-9,-9,0,...,-9,-9,-9,-9,0.4,-6.0,6.2,1022.5,1023.4,-9.0
8782,202412312200,113,320,6,-9,10000,-9,-9,-9,0,...,-9,-9,-9,-9,-0.7,-6.1,6.7,1022.7,1023.6,-9.0


In [313]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   YYMMDDHHMI  8784 non-null   int64  
 1   STN         8784 non-null   int64  
 2   WD          8784 non-null   int64  
 3   WS          8784 non-null   int64  
 4   GST         8784 non-null   int64  
 5   VS          8784 non-null   int64  
 6   RVR1        8784 non-null   int64  
 7   RVR2        8784 non-null   int64  
 8   WC          8784 non-null   int64  
 9   CA          8784 non-null   int64  
 10  CA1         8784 non-null   int64  
 11  CT1         8784 non-null   int64  
 12  CH1         8784 non-null   int64  
 13  CA2         8784 non-null   int64  
 14  CT2         8784 non-null   int64  
 15  CH2         8784 non-null   int64  
 16  CA3         8784 non-null   int64  
 17  CT3         8784 non-null   int64  
 18  CH3         8784 non-null   int64  
 19  CA4         8784 non-null  

* YYMMDD와 YYMMDDHHMI는 동일한 숫자형임. 그러나 자리수가 YYMMDD가 더 많으므로, 이를 맞춰준다

In [314]:
## int형식이니, 걍 100으로 나눠본다!
new_data["YYMMDD"] = new_data["YYMMDD"].apply(lambda x: int(pd.Series(x/100)))

C:\Users\user\AppData\Local\Temp\ipykernel_34764\3326424062.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  new_data["YYMMDD"] = new_data["YYMMDD"].apply(lambda x: int(pd.Series(x/100)))


In [315]:
# 심지어 시간/분까지의 단위위
new_data["YYMMDD"] = new_data[""]

KeyError: ''

In [ ]:
# 일단은 다음과 같이 만들어보자!
## 1. 리스트 변수를 하나 만든다
w_date = []

### 2. 'YYMMDD'의 열을 이용한 for 문을 만든다.(for x in data["YYMMDD"])
### 3. x가 


#### 데이터 전처리 과정 진행
- 범주형 : 라벨 인코딩 처리
- 숫자형 : 스케일링 진행

In [137]:
## Labelencoder부터 불러오기
from sklearn.preprocessing import LabelEncoder

## Label encoding을 적용할 변수 리스트 생성
features = ['airline', 'flightcode']

## 변환
for feature in features:
    le = LabelEncoder()
    le = le.fit(new_data[feature])
    new_data[feature] = le.transform(new_data[feature])

In [138]:
# 확인
new_data

,airline,flightcode,month,day,arrival_code,arrival_kor,planned_time,real_time,time_new,distance,delayed
0,34,267,1,1,DXB,두바이,5.040,5.040,20240101000500,6836,0
1,10,676,1,1,UBN,칭기즈 칸,5.040,1.008,20240101000100,1995,0
2,68,841,1,1,DOH,도하,14.976,50.976,20240101005100,7270,1
3,75,911,1,1,IST,이스탄불,14.976,18.000,20240101001800,8055,0
4,55,309,1,1,SYD,시드니,20.016,114.048,20240101015400,8320,1
...,...,...,...,...,...,...,...,...,...,...,...
179217,3,406,12,31,SIN,싱가포르,1390.032,1413.072,20241231233300,4630,1
179218,29,899,12,31,SIN,싱가포르,1395.072,1428.048,20241231234800,4630,1
179219,65,847,12,31,TAO,청도,1399.968,1406.016,20241231232600,620,0
179220,81,1064,12,31,MNL,마닐라,1419.984,1431.072,20241231235100,2610,0


##### 불필요한 columns 제거
- 1. 'arrival_code', 'arrival_kor'의 경우 distance 값으로 통일 가능
- 2. 'planned_time', 'real_time'의 경우 지연 여부를 위한 것이므로 삭제 필요

In [139]:
## 필요없는 columns drop
new_data = new_data.drop(["arrival_code", "arrival_kor", 'planned_time', 'real_time'], axis = 1)

In [140]:
new_data

,airline,flightcode,month,day,time_new,distance,delayed
0,34,267,1,1,20240101000500,6836,0
1,10,676,1,1,20240101000100,1995,0
2,68,841,1,1,20240101005100,7270,1
3,75,911,1,1,20240101001800,8055,0
4,55,309,1,1,20240101015400,8320,1
...,...,...,...,...,...,...,...
179217,3,406,12,31,20241231233300,4630,1
179218,29,899,12,31,20241231234800,4630,1
179219,65,847,12,31,20241231232600,620,0
179220,81,1064,12,31,20241231235100,2610,0
